In [1]:
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf

## Prepare data

In [3]:
har_x_train, har_y_train, list_ch_train = read_data(data_path="./dataset/UCI HAR Dataset/", split="train") # train
har_x_test, har_y_test, list_ch_test = read_data(data_path="./dataset/UCI HAR Dataset/", split="train") # test

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(har_x_train, har_y_train, stratify=har_y_train, random_state=42)

In [5]:
X_train.shape, y_train.shape

((5514, 128, 9), (5514,))

# Build DNN model

## Hyperparameters

In [6]:
batch_size = 60
seq_len = 128
learning_rate = 0.01
n_epochs = 100

n_classes = 7 # from 1 - 6
n_channels = 9

reshape all data into 1D

In [7]:
X_train.shape, y_train.shape

((5514, 128, 9), (5514,))

In [8]:
X_train = X_train.reshape(-1, seq_len * n_channels)
X_valid = X_valid.reshape(-1, seq_len * n_channels)
har_x_test = har_x_test.reshape(-1, seq_len * n_channels)

In [9]:
X_train.shape, y_train.shape

((5514, 1152), (5514,))

## the model

In [10]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, seq_len * n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")


In [11]:
with tf.name_scope("dnn"):
    
    dense1 = tf.layers.dense(X, 300, activation=tf.nn.relu, name="dense1")
    dense2 = tf.layers.dense(dense1, 100, activation=tf.nn.relu, name="dense2")
    dense3 = tf.layers.dense(dense2, 50, activation=tf.nn.relu, name="dense3")
    
    
    #fc1 = tf.layers.dense(drop, 100)
    #fc2 = tf.layers.dense(fc1, 100)
    logits = tf.layers.dense(dense3, n_classes)
    
    

In [12]:
logits.get_shape()

TensorShape([Dimension(None), Dimension(7)])

In [13]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [14]:
with tf.name_scope("train"):
    training_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [15]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [16]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [17]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for X_batch, y_batch in get_batches(X_train, y_train, batch_size):
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch})
        valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        print("Epoch: ", epoch, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_cnn_model.ckpt")

Epoch:  0 Valid accuracy:  0.861262
Epoch:  1 Valid accuracy:  0.908052
Epoch:  2 Valid accuracy:  0.875952
Epoch:  3 Valid accuracy:  0.934712
Epoch:  4 Valid accuracy:  0.942329
Epoch:  5 Valid accuracy:  0.915125
Epoch:  6 Valid accuracy:  0.92383
Epoch:  7 Valid accuracy:  0.930903
Epoch:  8 Valid accuracy:  0.930903
Epoch:  9 Valid accuracy:  0.920022
Epoch:  10 Valid accuracy:  0.87432
Epoch:  11 Valid accuracy:  0.926551
Epoch:  12 Valid accuracy:  0.945593
Epoch:  13 Valid accuracy:  0.951578
Epoch:  14 Valid accuracy:  0.949401
Epoch:  15 Valid accuracy:  0.949401
Epoch:  16 Valid accuracy:  0.945593
Epoch:  17 Valid accuracy:  0.949401
Epoch:  18 Valid accuracy:  0.949946
Epoch:  19 Valid accuracy:  0.947769
Epoch:  20 Valid accuracy:  0.949946
Epoch:  21 Valid accuracy:  0.942873
Epoch:  22 Valid accuracy:  0.921654
Epoch:  23 Valid accuracy:  0.927095
Epoch:  24 Valid accuracy:  0.922198
Epoch:  25 Valid accuracy:  0.945049
Epoch:  26 Valid accuracy:  0.942873
Epoch:  27 Va

In [18]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_cnn_model.ckpt")
    test_acc = accuracy.eval(feed_dict={X: har_x_test, y: har_y_test})
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_cnn_model.ckpt
Test Accuracy:  0.960827
